In [3]:
from langchain_community.llms import Ollama
from langchain import hub
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain.prompts import ChatPromptTemplate

In [4]:
model_name = "llama3"

llm = Ollama(model=model_name)
llm.invoke("Tell me a joke")

"Why couldn't the bicycle stand up by itself?\n\nBecause it was two-tired!\n\nHope that made you smile!"

In [5]:
loader = UnstructuredFileLoader("../data/semester-data.md")
docs = loader.load()
docs


[Document(page_content='Course Syllabus\n\nCourse Number:INST346\nCourse Title: Technology, Infrastructure and Architecture\nSemester: Summer 2024\nInstructor: Mr. Donal Heidenblad\nEmail: dheidenb@umd.edu\nOffice Hours: By Appointment through Calendly: https://calendly.com/dheidenb\n\nCourse Description\n\nExamines the basic concepts of computer hardware, systems software, networking, client/server architectures, cloud computing, distributed systems, and high performance computing as applied to information rich domains. Technology and architectures will be discussed within the contexts of solving social issues, supportingscience, and conducting business operations. Current computing topics such as web environments, IoT, security, management, and policy will also be reviewed.\n\nCourse Objectives\n\nAfter successfully completing this course, you will be able to:\n- Articulate major hardware, software and networking concepts and components that comprise current digital information infra

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs) 

vectorstore = Chroma.from_documents(documents=splits, embedding=OllamaEmbeddings(model=model_name))


In [7]:
retreiver = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [8]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [9]:
prompt = hub.pull("rlm/rag-prompt-llama")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt-llama', 'lc_hub_commit_hash': '693a2db5447e3b58c060a6ac02758dc7f1aaaaa4ee6214d127bf70b443158630'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="[INST]<<SYS>> You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.<</SYS>> \nQuestion: {question} \nContext: {context} \nAnswer: [/INST]"))])

In [10]:
rag_chain = (
    {"context": retreiver | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [11]:
for chunk in rag_chain.stream("Who is the professor that is teaching this class?"):
    print(chunk, end="", flush=True)

Number of requested results 6 is greater than number of elements in index 3, updating n_results = 3


The professor teaching this class is Mr. Donal Heidenblad.

In [12]:
for chunk in rag_chain.stream("When does this class meet?"):
    print(chunk, end="", flush=True)

Number of requested results 6 is greater than number of elements in index 3, updating n_results = 3


[/SYS] According to the Course Structure section, this class meets on Thursdays from 10:00-10:50 AM in room 0302N of Hornbake Library. [/SYS]